In [27]:
import cv2
import pandas as pd
import os
# https://github.com/AdrianNunez/Fall-Detection-with-CNNs-and-Optical-Flow

# Define directories and CSV file
fall_dir = "originals/fall"
root_output_dir = "OFCNN"
csv_file = "Labels.csv"

# Load the fall event data
fall_data = pd.read_csv(csv_file)
print(fall_data)

     Clip Fall Start Fall End
0       1      46.65    49.15
1       2       11.9     14.2
2       3       2.65        5
3       4        0.4      1.4
4       5        0.9     3.15
..    ...        ...      ...
115   116      START      2.1
116   117      START      END
117   118       1.15        2
118   119          1        2
119   120        2.9      END

[120 rows x 3 columns]


In [28]:
def calculate_optical_flow(prvs_frame, next_frame):
    """Calculates and returns the optical flow between two frames."""
    flow = cv2.calcOpticalFlowFarneback(
        prvs_frame, next_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0
    )
    # Convert the flow to HSV to create a color representation
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv = np.zeros_like(cv2.cvtColor(prvs_frame, cv2.COLOR_GRAY2BGR))
    hsv[..., 1] = 255
    hsv[..., 0] = ang * 180 / np.pi / 2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    optical_flow_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return optical_flow_image

In [29]:
import cv2
import pandas as pd
import os
import numpy as np
#Investige 13,26,27,37

# Create directories if they don't exist
os.makedirs(output_dir, exist_ok=True)

# Define processing parameters
clip_durations = [1.0, 2.0]  # clip lengths in seconds
slide_interval = 0.5  # sliding window in seconds

# Process each video in the fall directory
for video_file in os.listdir(fall_dir):
    if video_file.endswith(".mp4"):
        # Extract the clip number from the filename
        clip_number = int(video_file.split('_')[1])

        # Retrieve fall timing for this clip
        fall_row = fall_data[fall_data['Clip'] == clip_number]

        fall_start = fall_row.iloc[0, 1]
        fall_end = fall_row.iloc[0, 2]

        # Handle "START" and "END" values
        if fall_start == "START":
            fall_start = 0.0
        else:
            fall_start = float(fall_start)

        video_path = os.path.join(fall_dir, video_file)
        cap = cv2.VideoCapture(video_path)
        """
        Majority of clips are natively 25 fps a few are lower 13,26,27,37.
        Fixing the fps for a consistent stack of frames for learning.
        """
        fps = 25 
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        video_duration = total_frames / fps

        if fall_end == "END":
            fall_end = float(video_duration)
        else:
            fall_end = float(fall_end)

        # Iterate over each clip duration
        for duration in clip_durations:
            # Reopen the video to reset position for each duration
            cap = cv2.VideoCapture(video_path)
            frame_count = int(duration * fps)  # frames per clip
            slide_frames = int(slide_interval * fps)

            clip_index = 0  # counter for clips
            start_frame = 0  # initial start frame

            while start_frame + frame_count <= total_frames:
                # Check if the clip contains a fall
                clip_start_time = float(start_frame / fps)
                clip_end_time = float((start_frame + frame_count) / fps)
                has_fall = not (clip_end_time <= fall_start or clip_start_time >= fall_end)
                label = "FALL" if has_fall else "NO"
                clip_dir = os.path.join(output_dir, str(duration))
                clip_dir = os.path.join(clip_dir, f"{clip_number:03}_{clip_index}_{label}")
                os.makedirs(clip_dir, exist_ok=True)

                # Initialize variables for optical flow
                cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
                ret, first_frame = cap.read()
                if not ret:
                    break
                
                # Write out first frame
                prvs_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
                frame_img_name = f"frame_{clip_index:03}_0.png"
                cv2.imwrite(os.path.join(clip_dir, frame_img_name), first_frame)

                # Process remaining frames
                for i in range(1, frame_count):
                    ret, next_frame = cap.read()
                    if not ret:
                        break

                    # Save the current frame
                    frame_img_name = f"frame_{clip_index:03}_{i}.png"
                    cv2.imwrite(os.path.join(clip_dir, frame_img_name), next_frame)

                    # Convert the current frame to grayscale for optical flow
                    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

                    # Calculate optical flow image
                    optical_flow_image = calculate_optical_flow(prvs_gray, next_gray)
                    
                    # Save the optical flow image
                    flow_img_name = f"flow_{clip_index:03}_{i}.png"
                    cv2.imwrite(os.path.join(clip_dir, flow_img_name), optical_flow_image)

                    # Update previous frame
                    prvs_gray = next_gray

                print(f"Saved frames and optical flow images for clip: {clip_number:03}_{clip_index}")

                # Update frame index for sliding window
                start_frame += slide_frames
                clip_index += 1

            cap.release()


Saved frames and optical flow images for clip: 002_0
Saved frames and optical flow images for clip: 002_1
Saved frames and optical flow images for clip: 002_2
Saved frames and optical flow images for clip: 002_3
Saved frames and optical flow images for clip: 002_4
Saved frames and optical flow images for clip: 002_5
Saved frames and optical flow images for clip: 002_6
Saved frames and optical flow images for clip: 002_7
Saved frames and optical flow images for clip: 002_8
Saved frames and optical flow images for clip: 002_9
Saved frames and optical flow images for clip: 002_10
Saved frames and optical flow images for clip: 002_11
Saved frames and optical flow images for clip: 002_12
Saved frames and optical flow images for clip: 002_13
Saved frames and optical flow images for clip: 002_14
Saved frames and optical flow images for clip: 002_15
Saved frames and optical flow images for clip: 002_16
Saved frames and optical flow images for clip: 002_17
Saved frames and optical flow images f